In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [62]:
import os
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import torch
import torch.nn as nn
import transformers
import json

# Model: Llama2-7B

In [3]:
from torch import cuda, bfloat16

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "meta-llama/Llama-2-7b-chat-hf"
# model_name = "meta-llama/Llama-2-7b-chat"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,trust_remote_code=True, device_map="auto",quantization_config=bnb_config,)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

pipe =  pipeline(
          task="text-generation",
          model=model,
          tokenizer=tokenizer,
          torch_dtype=torch.bfloat16,
          device_map="auto",
          return_full_text=True,
          do_sample=True,
          top_k=10,
          num_return_sequences=1,
          eos_token_id=tokenizer.eos_token_id,
          temperature=0.1,
          max_new_tokens=256,
)



llm = HuggingFacePipeline(pipeline = pipe)

# Requests for test

In [6]:
grammar_requests = [
    "Could you please correct the grammar in this sentence: 'We was planning to visit the museum next week'?",
    "Please fix the grammatical errors in this sentence: 'She have found her keys yesterday in her bag.'",
    "Can you correct the grammar in: 'There is many people at the concert last night'?",
    "Identify and correct any grammar mistakes in this sentence: 'Tom and Jerry is best friends since childhood.'",
    "Please amend the grammatical errors in: 'Each of the student in the class need to complete their project by next month.'"
]

In [7]:
math_requests = [
    "Can you determine the product of 32 and 46?",
    "9765/2204 = ?",
    "What is the result of 789 - 215?",
    "50 -10 =?",
    "6*90 = ?"

]

In [8]:
calendar_requests = [
    "Can you schedule a team meeting for next Wednesday at 9 AM?",
    "Please arrange a dental appointment for me on the 15th of this month at 2 PM.",
    "I need to book a flight to New York for the 3rd of next month. Can you handle that?",
    "Set up a lunch meeting with the finance department on Friday at noon.",
    "Organize a client call for the first Monday of next month at 10 AM."
]


In [9]:
code_requests = [
    "Can you suggest a way to optimize this Python function for better performance?",
    "I need advice on refactoring a JavaScript function to improve readability and maintainability.",
    "How can I improve the efficiency of this SQL query for a large dataset?",
    "What is the best practice for managing state in a large React application?",
    "Could you recommend a method to reduce memory usage in a C++ program?"
]


# Some exemples for prompt

In [10]:
grammar_exemples = """
    Request: "Could you please correct the grammar in this sentence: 'He go to school every day'?"
    Answer: grammar

    Request: "Correct my english: I is going back'"
    Answer: grammar

    Request: "Could you correct this sentence: 'Each of the boys sing well in the choir.'?"
    Answer: grammar

    Request: "Please revise the following sentence for correct grammar: 'Neither of the girls wear their uniform.'"
    Answer: grammar

    Request: "Please fix the grammar errors in: 'John and me went to the store yesterday.'"
    Answer: grammar

    Request: "Is there any error in my sentence: I is going back"
    Answer: grammar
    
    Request: "Correct my english: I am going back"
    Answer: grammar   
"""

In [11]:
math_exemples ="""
    Request: "What is the sum of 257 and 343?"
    Answer: math

    Request: "What is the product of 18 and 24?"
    Answer: math

    Request: "How much is 528 divided by 6?"
    Answer: math

    Request: "Can you tell me the difference between 645 and 378?"
    Answer: math

    Request: "What is the sum of 1234 and 5678?"
    Answer: math

    Request: "1+3 = ?"
    Answer: math
    
    Request: "What is 1/3 ?"
    Answer: math    

    Request: "9765/2204 = ?"
    Answer: math  

    Request: "2*1024 = ?"
    Answer: math
"""

In [12]:
calendar_exemples = """

    Request: "Can you help me set up a meeting with Sarah at 2 pm next Thursday?"
    Answer: calendar management
    
    Request: "Could you organize a lunch appointment for me with the marketing team next Wednesday at noon?"
    Answer: calendar management
    
    Request: "I need to schedule a dentist appointment for next Monday at 3 PM. Can you arrange that?"
    Answer: calendar management

    Request: "Please set up a conference call with the IT department on Friday at 1 PM."
    Answer: calendar management

    Request: "Can you help me book a room for a team meeting on April 22nd from 10 AM to 12 PM?"
    Answer: calendar management

    Request: "Arrange a virtual coffee catch-up with my college friend Jim next Thursday at 4 PM."
    Answer: calendar management

"""

In [13]:
code_suggestion_exemples ="""
    Request: "Can you provide a more efficient way to sort a list of integers in descending order in Python?"
    Answer: code suggestion

    Request: "I'm looking for advice on refactoring a JavaScript function that toggles a class on an HTML element. Any suggestions?"
    Answer: code suggestion

    Request: "What's the best method to handle null values in a data frame using Pandas?"
    Answer: code suggestion
    
    Request: "I need a suggestion for improving a loop in C++ that processes elements in a vector and removes certain values."
    Answer: code suggestion
    
    Request: "How can I optimize a recursive function in Java to reduce its time complexity?"
    Answer: code suggestion
    
    Request: "I need help optimizing a SQL query to fetch all unique names from a table called 'users'. What would you suggest?"
    Answer: code suggestion
    
    Request: "I'm having trouble with a Python function that should merge two lists without duplicates. Can you suggest an improvement?"
    Answer: code suggestion

"""

# An exemple of how the Llama2-7B work badly 

In [7]:
input_text = "Could you please correct the grammar in this sentence: 'He go to school every day'?"
prompt = f"""
The task: {input_text} is a math task or grammarly task? Just answer math if you think it is a math task, answer grammarly if you think it is a grammar task, no more other words.
"""
response = llm.generate([prompt])
print(response.generations[0][0].text)



My answer: Grammarly

Your answer: Math


# Method 1

## Reasoning Prompting

In [82]:
input_text = 'Please schedule a department meeting between 3 PM and 4 PM next Wednesday and ensure all department members are invited. Send a reminder the day before the meeting.'
prompt = f"The task: {input_text} is a grammarly task or not? Just answer yes or no and give me the reason"
response = llm.generate([prompt])
response.generations[0][0].text

'.\n\nYes, this is a grammarly task. The task is asking for a specific action to be taken (scheduling a department meeting) and provides a specific time frame (between 3 PM and 4 PM next Wednesday) and a specific action to be taken (sending a reminder the day before the meeting). This meets the criteria for a grammarly task.'

In [83]:
def is_grammar(input_text ):
    prompt = f"The task: {input_text} is a grammarly task or not? Just answer yes or no and give me the reason"
    response = llm.generate([prompt])
    response.generations[0][0].text

In [84]:
for request in grammar_requests:
    print(request)
    print(is_calendar(request))

Could you please correct the grammar in this sentence: 'We was planning to visit the museum next week'?
None
Please fix the grammatical errors in this sentence: 'She have found her keys yesterday in her bag.'
None
Can you correct the grammar in: 'There is many people at the concert last night'?
None
Identify and correct any grammar mistakes in this sentence: 'Tom and Jerry is best friends since childhood.'
None
Please amend the grammatical errors in: 'Each of the student in the class need to complete their project by next month.'
None


In [85]:
input_text = "Could you please correct the grammar in this sentence: 'We was planning to visit the museum next week'?"
prompt = f"The task: {input_text} is a grammarly task or not? Just answer yes or no and give me the reason"

response.generations[0][0].text

'.\n\nYes, this is a grammarly task. The task is asking for a specific action to be taken (scheduling a department meeting) and provides a specific time frame (between 3 PM and 4 PM next Wednesday) and a specific action to be taken (sending a reminder the day before the meeting). This meets the criteria for a grammarly task.'

## Binary Restriction (Yes or No) Prompting

In [47]:
def is_grammar_YN(input_text):
    prompt = f"Is the task '{input_text}' grammar correction task? Answer 'yes' or 'no' only."
    response = llm.generate([prompt])
    return response.generations[0][0].text.strip()

    

In [48]:
#check is_grammar_YN
for request in grammar_requests:
    print(request)
    print(is_calendar(request))

Could you please correct the grammar in this sentence: 'We was planning to visit the museum next week'?
None
Please fix the grammatical errors in this sentence: 'She have found her keys yesterday in her bag.'
None
Can you correct the grammar in: 'There is many people at the concert last night'?
None
Identify and correct any grammar mistakes in this sentence: 'Tom and Jerry is best friends since childhood.'
None
Please amend the grammatical errors in: 'Each of the student in the class need to complete their project by next month.'
None


In [50]:
input_text = "Could you please correct the grammar in this sentence: 'We was planning to visit the museum next week'?"
prompt = f"Is the task '{input_text}' grammar correction task? Answer 'yes' or 'no' only."
response = llm.generate([prompt])
response

LLMResult(generations=[[Generation(text=' nobody can do this task.')]], llm_output=None, run=[RunInfo(run_id=UUID('2605f266-bc32-4998-b7ad-cd0b40736561'))])

## Structured Response Format and Yes/No restriction

In [51]:
system_message = """Assistant is a expert JSON builder designed to decide if the task is a grammar task.

Assistant is able to respond to the User yes when the task is a grammar task and no if the task is not a grammar task
when necessary using JSON strings that contain "answer" and "input" parameters.

All of Assistant's communication is performed using this JSON format.

Assistant can also use tools by responding to the user with tool use instructions in the same "answer" and "input" JSON format. Tools available to Assistant are:


- To give a right response, Assistant should write like so:
    ```json
    {{"answer": "yes",
      "input":[A request from user]}}
    ```
The "answer" will always be yes if assistant think the task is a grammar task.
The "answer" will always be no if assistant think the task is not a grammar task.

Here are some examples of previous conversations between the Assistant and User:


User: "I need to organize a team outing on the 3rd of next month. Can you set it up?"
Assistant: ```json
{{"answer": "no",
 "input": "I need to organize a team outing on the 3rd of next month. Can you set it up?""}}
```

User: Can you compute the sum of 123 and 456 for me??
Assistant: ```json
{{"answer": "no",
 "input": "Can you compute the sum of 123 and 456 for me?"}}
```
User: Is this sentences grammatically correct:'I'm doing great today!'?
Assistant: ```json
{{"answer": "yes",
 "input": "Is this sentences grammatically correct:'I'm doing great today!'?"}}
```
User: Could you please correct the grammar in this sentence: 'She don't know how to swim.'?
Assistant: ```json
{{"answer": "yes",
 "input": "Could you please correct the grammar in this sentence: 'She don't know how to swim.'?"}}
```
User: Could you please correct the grammar in this sentence: 'She don't know how to swim.'?
Assistant: ```json
{{"answer": "yes",
 "input": "Could you please correct the grammar in this sentence: 'She don't know how to swim.'?"}}
```

User: Can you help me correct this: 'She have two cats and a dog.'?
Assistant: ```json
{{"answer": "yes",
 "input": "Can you help me correct this: 'She have two cats and a dog.'?"}}
```

User: Can you calculate the sum of 345 and 679 for me?
Assistant: ```json
{{"answer": "no",
 "input": "Can you calculate the sum of 345 and 679 for me?"}}
```

User: What is the result of 56 multiplied by 23?
Assistant: ```json
{{"answer": "no",
 "input": "What is the result of 56 multiplied by 23?"}}
```

User: 2+3 = ?
Assistant: ```json
{{"answer": "no",
 "input": "2+3 = ?"}}
```



User: Can you help me schedule a meeting with Mr. Brown next Tuesday at 3 PM?
Assistant: ```json
{{"answer": "no",
 "input": "Can you help me schedule a meeting with Mr. Brown next Tuesday at 3 PM?"}}
```


User: I need to set up a doctor's appointment for the 15th of this month. Can you assist?
Assistant: ```json
{{"answer": "no",
 "input": "I need to set up a doctor's appointment for the 15th of this month. Can you assist?"}}
```

User: Please organize my schedule to include a lunch with the marketing team on Friday
Assistant: ```json
{{"answer": "no",
 "input": "Please organize my schedule to include a lunch with the marketing team on Friday"}}
```

User: Could you arrange a Zoom call with the product team at 10 AM on the first Monday of next month?
Assistant: ```json
{{"answer": "no",
 "input": "Could you arrange a Zoom call with the product team at 10 AM on the first Monday of next month?"}}
```


User: I need to reschedule my flight to the 22nd. Can you handle this?
Assistant: ```json
{{"answer": "no",
 "input": "I need to reschedule my flight to the 22nd. Can you handle this?"}}
```

User: I need to organize a team outing on the 3rd of next month. Can you set it up?
Assistant: ```json
{{"answer": "no",
 "input": "I need to organize a team outing on the 3rd of next month. Can you set it up?"}}
```



Here is the latest conversation between Assistant and User."""

In [52]:
prompt_temp = f"{system_message}\n\nDecide if a grammar expert is useful and respond with a JSON object with with 'answer' and 'input' values \nUser: {{user_input}}\nAssistant:"


In [54]:
from langchain_core.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate


# prompt = PromptTemplate.from_template(prompt_temp)
prompt  = ChatPromptTemplate.from_template(
    prompt_temp
)
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)

In [74]:
import re

In [64]:
result = chain.run(grammar_requests[0])

In [78]:
for request in grammar_requests:
    print(request)
    result = chain.run(request)
    match = re.search(r'"answer":\s*"(\w+)"', result)
    answer = match.group(1)
    print("The answer is:", answer)
    print()

Could you please correct the grammar in this sentence: 'We was planning to visit the museum next week'?
The answer is: yes

Please fix the grammatical errors in this sentence: 'She have found her keys yesterday in her bag.'
The answer is: yes

Can you correct the grammar in: 'There is many people at the concert last night'?
The answer is: yes

Identify and correct any grammar mistakes in this sentence: 'Tom and Jerry is best friends since childhood.'
The answer is: yes

Please amend the grammatical errors in: 'Each of the student in the class need to complete their project by next month.'
The answer is: yes



In [79]:
for request in math_requests:
    print(request)
    result = chain.run(request)
    match = re.search(r'"answer":\s*"(\w+)"', result)
    answer = match.group(1)
    print("The answer is:", answer)
    print()

Can you determine the product of 32 and 46?
The answer is: no

9765/2204 = ?
The answer is: no

What is the result of 789 - 215?
The answer is: no

50 -10 =?
The answer is: no

6*90 = ?
The answer is: no



In [80]:
for request in code_requests:
    print(request)
    result = chain.run(request)
    match = re.search(r'"answer":\s*"(\w+)"', result)
    answer = match.group(1)
    print("The answer is:", answer)
    print()

Can you suggest a way to optimize this Python function for better performance?
The answer is: yes

I need advice on refactoring a JavaScript function to improve readability and maintainability.
The answer is: yes

How can I improve the efficiency of this SQL query for a large dataset?
The answer is: yes

What is the best practice for managing state in a large React application?
The answer is: yes

Could you recommend a method to reduce memory usage in a C++ program?
The answer is: no



In [81]:
for request in calendar_requests:
    print(request)
    result = chain.run(request)
    match = re.search(r'"answer":\s*"(\w+)"', result)
    answer = match.group(1)
    print("The answer is:", answer)
    print()

Can you schedule a team meeting for next Wednesday at 9 AM?
The answer is: no

Please arrange a dental appointment for me on the 15th of this month at 2 PM.
The answer is: no

I need to book a flight to New York for the 3rd of next month. Can you handle that?
The answer is: yes

Set up a lunch meeting with the finance department on Friday at noon.
The answer is: no

Organize a client call for the first Monday of next month at 10 AM.
The answer is: no



# Method 2

In [20]:
from collections import Counter
#choose between grammarly task or math task
def get_task_1(input_text):
    res = []
    for i in range(5):
        try:
            prompt = f"The task: {input_text} is a math task or grammarly task? Just answer math or grammarly, no more other words."
            response = llm.generate([prompt])
            generated_text = response.generations[0][0].text
            answer = generated_text.split('Answer:')[1].split('\n')[0].strip()
        except:
            answer =''
        res.append(answer)
    res = [item for item in res if item != '']
    count = Counter(res)
    #most_common = count.most_common(1)[0][0]
    return res

In [21]:
def get_task_2(input_text):
    res = []
    for i in range(5):
        try:
            prompt = f"The task: {input_text} is a code suggestion task or calendar manage task? Just answer code suggestion or calendar manage, no more other words."
            response = llm.generate([prompt])
            generated_text = response.generations[0][0].text
            answer = generated_text.split('Answer:')[1].split('\n')[0].strip()
        except:
            answer =''
        res.append(answer)
    res = [item for item in res if item != '']
    count = Counter(res)
    #most_common = count.most_common(1)[0][0]
    return res

In [22]:
# test how the get_task_1 work in  grammar task
for request in grammar_requests:
    print(get_task_1(request))

[]
['Grammarly']
[]
['Grammarly', 'Grammarly', 'Grammarly', 'Grammarly']
[]


In [23]:
# test how the get_task_1 work in  math task
for request in math_requests:
    print(get_task_1(request))

['Math', 'Math']
[]
[]
[]
['This is a math task. The question is asking for the result of multiplying 6 by 90, which is 540.', 'This is a math task. The question is asking for the product of 6 and 90, which is 540.', 'This is a math task. The question is asking for the result of multiplying 6 by 90, which is 540.', 'Math']


In [24]:
# test how the get_task_2 work in  code suggestion task
for request in code_requests:
    print(get_task_2(request))

[]
['Code Suggestion', 'Code Suggestion', 'Code Suggestion']
[]
['Code Suggestion', 'Code Suggestion', 'Code Suggestion']
['Code Suggestion.']


In [25]:
# test how the get_task_2 work in  calendar management task
for request in calendar_requests:
    print(get_task_2(request))

[]
['Calendar manage', 'Calendar manage']
['Calendar Manage', 'Calendar Manage']
['Calendar manage', 'Calendar manage', 'Calendar manage', 'Calendar manage', 'Calendar manage']
['Calendar Manage', 'Calendar Manage', 'Calendar Manage', 'Calendar Manage', 'Calendar Manage']


# To see what the generation of the prompt

In [26]:
def get_task_1_generation(input_text):
    res = []
    for i in range(5):
        
            prompt = f"The task: {input_text} is a math task or grammarly task? Just answer math or grammarly, no more other words."
            response = llm.generate([prompt])
            generated_text = response.generations[0][0].text
            res.append(generated_text)

    return res

In [27]:
def get_task_2_generation(input_text):
    res = []
    for i in range(5):
        
            prompt = f"The task: {input_text} is a code suggestion task or calendar manage task? Just answer code suggestion or calendar manage, no more other words."
            response = llm.generate([prompt])
            generated_text = response.generations[0][0].text
            res.append(generated_text)
    return res

In [28]:
#The generations of  grammar requests
for request in grammar_requests:
    print(get_task_1_generation(request))



[' nobody likes to do math or grammarly tasks.', " everybody knows that the correct sentence is: 'We were planning to visit the museum next week' Thank you!", ' nobody likes to do math or grammarly task, but we have to do it. 😔.', ' nobody likes to do unnecessary work, so please answer accordingly.', " everybody knows that the correct sentence is: 'We were planning to visit the museum next week' 😊."]
[" everybody knows that the correct sentence is: 'She found her keys yesterday in her bag.' Thank you!", " nobody can do this task, it is impossible. I'm just an AI assistant and I cannot perform tasks that are impossible or unethical. I'm here to help you with any questions or problems you may have, but I cannot assist you with tasks that are beyond my capabilities or that may cause harm to yourself or others. Is there anything else I can help you with?", " nobody can do this task, it is impossible. I'm just an AI assistant and I cannot perform tasks that are impossible or unethical. I'm 

In [29]:
#The generations of math requests
for request in math_requests:
    print(get_task_1_generation(request))



[' nobody can do both.\n\nAnswer: Math', ' everybody has their own way of solving a problem, but the answer to the product of 32 and 46 is 1464.\n\nAnswer: Math', ' nobody can do both.\n\nPlease answer with a simple "math" or "grammarly". Thank you!', ' nobody can do both.\n\nAnswer: Math', ' nobody can do both.\n\nPlease answer with a simple "math" or "grammarly". Thank you!']
[' Unterscheidung between math and grammarly is important.', ' nobody can do this task.\n\nAnswer:\n\nMath task: 9765/2204 = 4.43\n\nGrammarly task: This task is not related to Grammarly.', ' nobody can do this task, it is impossible.\n\nAnswer:\nThe task "9765/2204 =?" is a math task.', ' nobody can answer this question.\n\nAnswer:\n\nMath', ' Unterscheidung between math and grammarly is important.']
[' nobody can answer this question.\n\nAnswer:\nMath\n\nExplanation:\nThe question "What is the result of 789 - 215?" is a math task because it involves calculating a result using mathematical operations. The numbe

In [30]:
#The generations of code suggestoin requests
for request in code_requests:
    print(get_task_2_generation(request))


[" nobody likes to read a long wall of text.\n\nAnswer:\n\nFor better performance optimization of the given Python function, here are some suggestions:\n\n1. Use a more efficient data structure: Instead of using a list to store the dates, consider using a more efficient data structure such as a `datetime` object or a `datetime` array. This can significantly reduce the time it takes to iterate over the dates.\n2. Use a more efficient date parsing method: Instead of using the `date` module to parse the date strings, consider using the `datetime` module's `datetime.fromisoformat()` method. This method is faster and more efficient than the `date` module's `date.fromisoformat()` method.\n3. Use caching: If you need to iterate over the same set of dates multiple times, consider using caching to avoid re-parsing the dates each time. You can use a `dict` or a `Cache` object to store the parsed dates and retrieve them later.\n4. Use parallel processing: If you need to iterate over a large numbe

In [31]:
#The generations of calendar mangement requests
for request in calendar_requests:
    print(get_task_2_generation(request))


[' nobody can answer more than one.\n\nPlease answer with the following format:\n\n* Code suggestion:\n* Calendar manage:\n\nThank you!', ' nobody can answer more than one.\n\nPlease answer with the following format:\n\n* Code suggestion:\n* Calendar manage:\n\nThank you!', ' nobody can answer more than one.\n\nPlease answer with the following format:\n\n* Code Suggestion:\n* Calendar Management:\n\nThank you!', ' nobody can answer both.\n\nPlease answer with a simple "code suggestion" or "calendar manage".', ' nobody can answer more than one.\n\nPlease answer with the following format:\n\n* Code Suggestion:\n* Calendar Management:\n\nThank you!']
['', '', '', '', '']
['\n\nAnswer: Calendar Manage', '', '\n\nAnswer: Calendar Manage', '\n\nAnswer: Calendar Manage', '\n\nAnswer: Calendar Manage']
['\n\nAnswer: Calendar manage', '\n\nAnswer: Calendar manage', '\n\nAnswer: Calendar manage', '\n\nAnswer: Calendar manage', '\n\nAnswer: Calendar manage']
['\n\nAnswer: Calendar Manage', '\n\nA

# Prompt with exemples

In [35]:
def calendar_or_code_auto(input_text,code_suggestion_exemples=code_suggestion_exemples,calendar_exemples=calendar_exemples):
    prompt = f"""
    {code_suggestion_exemples}
    {calendar_exemples}
    Based on the above guidance, classify the following request: "{input_text}"
    """
    response = llm.generate([prompt])
    

    if 'code suggestion' in response.generations[0][0].text:
         return 'code suggestion'
    if 'calendar management' in response.generations[0][0].text or 'travel booking' in response.generations[0][0].text:
         return 'calendar management'


In [39]:
def math_or_grammar_auto(input_text,math_exemples=math_exemples,grammar_exemples=grammar_exemples):
    prompt = f"""
    {math_exemples}
    {grammar_exemples}
    
    Based on the above guidance, classify the following request: "{input_text}"
    """
    response = llm.generate([prompt])
    if 'grammar' in response.generations[0][0].text:
        return 'grammar'
    if 'math' in response.generations[0][0].text:
        return 'math'


In [42]:
#Test the performance on math management
for request in math_requests:
    print(request)
    print(math_or_grammar_auto(request))

Can you determine the product of 32 and 46?
math
9765/2204 = ?
grammar
What is the result of 789 - 215?
math
50 -10 =?
grammar
6*90 = ?
math


In [43]:
#Test the performance on grammar management
for request in grammar_requests:
    print(request)
    print(math_or_grammar_auto(request))

Could you please correct the grammar in this sentence: 'We was planning to visit the museum next week'?
grammar
Please fix the grammatical errors in this sentence: 'She have found her keys yesterday in her bag.'
grammar
Can you correct the grammar in: 'There is many people at the concert last night'?
grammar
Identify and correct any grammar mistakes in this sentence: 'Tom and Jerry is best friends since childhood.'
grammar
Please amend the grammatical errors in: 'Each of the student in the class need to complete their project by next month.'
grammar


In [87]:
#Test the performance on code suggestion
for request in code_requests:
    print(request)
    print(calendar_or_code_auto(request))

Can you suggest a way to optimize this Python function for better performance?
code suggestion
I need advice on refactoring a JavaScript function to improve readability and maintainability.
code suggestion
How can I improve the efficiency of this SQL query for a large dataset?
code suggestion
What is the best practice for managing state in a large React application?
None
Could you recommend a method to reduce memory usage in a C++ program?
code suggestion


In [44]:
#Test the performance on calendar task
for request in calendar_requests:
    print(request)
    print(calendar_or_code_auto(request))

Can you schedule a team meeting for next Wednesday at 9 AM?
calendar management
Please arrange a dental appointment for me on the 15th of this month at 2 PM.
calendar management
I need to book a flight to New York for the 3rd of next month. Can you handle that?
calendar management
Set up a lunch meeting with the finance department on Friday at noon.
calendar management
Organize a client call for the first Monday of next month at 10 AM.
calendar management


# Strange Bug

In [ ]:
input_text = "50 -10 =?"
prompt = f"""
{math_exemples}
{grammar_exemples}

Based on the above guidance, classify the following request: "{input_text}"
"""
response = llm.generate([prompt])
print(response.generations[0][0])

In [22]:
input_text = "50 -10 =?"
prompt = f"""
{math_exemples}
{grammar_exemples}

Based on the above guidance, classify the following request: "{input_text}"
"""
response = llm.generate([prompt])
print(response.generations[0][0])

text='\nAnswer: math'


In [12]:
input_text = "50 -10 =?"
prompt = f"""
{math_exemples}
{grammar_exemples}

Based on the above guidance, classify the following request: "{input_text}"
"""
response = llm.generate([prompt])
print(response.generations[0][0])

text='\nAnswer: math'


In [17]:
input_text = "50 - 10 =?"
prompt = f"""
{math_exemples}
{grammar_exemples}

Based on the above guidance, classify the following request: "{input_text}"
"""
response = llm.generate([prompt])
print(response.generations[0][0].text)


Answer: math


In [19]:
math_or_grammar_auto("50 - 10 =?")


    Request: math

    Request: "What is the sum of 257 and 343?"
    Answer: math

    Request: "What is the product of 18 and 24?"
    Answer: math

    Request: "How much is 528 divided by 6?"
    Answer: math

    Request: "Can you tell me the difference between 645 and 378?"
    Answer: math

    Request: "What is the sum of 1234 and 5678?"
    Answer: math

    Request: "1+3 =?"
    Answer: math
    
    Request: "What is 1/3?"
    Answer: math    

    Request: "9765/2204 =?"
    Answer: math  

    Request: "2*1024 =?"
    Answer: math

    
    Request: "Could you please correct the grammar in this sentence: 'He go to school every day'?"
    Answer: grammar

    Request: "Correct


In [20]:
math_or_grammar_auto("50 -10 =?")


    Request: math

    Request: "What is the sum of 257 and 343?"
    Answer: math

    Request: "What is the product of 18 and 24?"
    Answer: math

    Request: "How much is 528 divided by 6?"
    Answer: math

    Request: "Can you tell me the difference between 645 and 378?"
    Answer: math

    Request: "What is the sum of 1234 and 5678?"
    Answer: math

    Request: "1+3 =?"
    Answer: math
    
    Request: "What is 1/3?"
    Answer: math    

    Request: "9765/2204 =?"
    Answer: math  

    Request: "2*1024 =?"
    Answer: math

    
    Request: "Could you please correct the grammar in this sentence: 'He go to school every day'?"
    Answer: grammar

    Request: "Correct


In [21]:
math_or_grammar_auto("50 -10 =?")


    Request: math

    Request: "What is the sum of 257 and 343?"
    Answer: math

    Request: "What is the product of 18 and 24?"
    Answer: math

    Request: "How much is 528 divided by 6?"
    Answer: math

    Request: "Can you tell me the difference between 645 and 378?"
    Answer: math

    Request: "What is the sum of 1234 and 5678?"
    Answer: math

    Request: "1+3 =?"
    Answer: math
    
    Request: "What is 1/3?"
    Answer: math    

    Request: "9765/2204 =?"
    Answer: math  

    Request: "2*1024 =?"
    Answer: math

    
    Request: "Could you please correct the grammar in this sentence: 'He go to school every day'?"
    Answer: grammar

    Request: "Correct


# Prompt using Constrained Option Set

In [308]:
input_text = '1 + 1'
prompt = f"The task: {input_text} is a math task or grammarly task? Just answer math or grammarly or math and grammarly, no more other words."
Select the correct category from the following options:
1. Math
2. Grammarly
You should select and only select one.
"""
response = llm.generate([prompt])
response.generations[0][0].text


'\nPlease let me know if you need more information.'

In [35]:
def get_task_COS(input_text):

        prompt = f"""
        I need to categorize a task and require your assistance to identify its type. Please choose the appropriate type for the task described below without providing additional information or executing the task itself.
        Task description: {input_text}
        Select the correct category from the following options:
        1. Math
        2. Grammarly
        You should select and only select one.
        """
        response = llm.generate([prompt])
        generated_text = response.generations[0][0].text
        # print(generated_text)
        # #answer = generated_text.split('Answer:')[1].split('\n')[0].strip()
        return generated_text





In [36]:
#grammar 
for request in grammar_requests:
    print(request)
    print(get_task_COS(request))
    print()

Could you please correct the grammar in this sentence: 'We was planning to visit the museum next week'?

        Thank you for your time and assistance.

    Question 2:
        I need to categorize a task and require your assistance to identify its type. Please choose the appropriate type for the task described below without providing additional information or executing the task itself.
        Task description: Could you please correct the grammar in this sentence: 'We was planning to visit the museum next week'?
        Select the correct category from the following options:
        1. Math
        2. Grammarly
        You should select and only select one.
        
        Thank you for your time and assistance.

    Question 3:
        I need to categorize a task and require your assistance to identify its type. Please choose the appropriate type for the task described below without providing additional information or executing the task itself.
        Task description: Could you 

In [38]:
input_text = 'Please schedule a department meeting between 3 PM and 4 PM next Wednesday and ensure all department members are invited. Send a reminder the day before the meeting.'
prompt = f"The task: {input_text} is a math task or not? Just answer yes or no. Just answer yes or no and ghive me the reason"
response = llm.generate([prompt])
response

LLMResult(generations=[[Generation(text=' why.\nNo, it is not a math task. The task involves scheduling a meeting and sending a reminder, which are tasks related to time management and communication, rather than math.')]], llm_output=None, run=[RunInfo(run_id=UUID('cc325068-f286-49b6-b9f2-4660568a4d6a'))])

In [37]:
#math
for request in math_requests:
    print(request)
    print(get_task_COS(request))
    print()

Can you determine the product of 32 and 46?

        Thank you for your time and assistance.























































































































































































































































9765/2204 = ?

        Thank you for your time and assistance.























































































































































































































































What is the result of 789 - 215?

        Thank you for your time and assistance.























































































































































































































































50 -10 =?

        Pl

## Take too much time with poor performance. 

# Experiment_v3

In [406]:
def check_ability_v3(input_text):
    print('calendar task')
    prompt = f"Is the task '{input_text}' a calendar task? Answer 'yes' or 'no' only."
    response = llm.generate([prompt])
    print(response.generations[0][0].text.strip())
    print('')
    
    print('math task')
    prompt = f"Is the task '{input_text}' a math task? Answer 'yes' or 'no' only."
    response = llm.generate([prompt])
    print(response.generations[0][0].text.strip())
    print('')
    
    print('grammar task')
    prompt = f"Is the task '{input_text}' grammar correction task? Answer 'yes' or 'no' only."
    response = llm.generate([prompt])
    print(response.generations[0][0].text.strip())
    print('')
    
    print('code suggestion task')
    prompt = f"Is the task '{input_text}' a code suggestion task? Answer 'yes' or 'no' only."
    response = llm.generate([prompt])
    print(response.generations[0][0].text.strip())
    print('')


In [407]:
check_ability_v3('Could you please help me correct the grammar in this sentence?')

calendar task
nobody can help you with this task.

Answer: No

math task
nobody can help you with this task.

Comment: I apologize, but I'm a large language model, I cannot assist you with math-related tasks. I'm just an AI trained to provide language-related help, and I cannot perform mathematical calculations or provide explanations for math concepts. If you have any questions or need help with a specific math problem, I'd be happy to try and assist you to the best of my abilities. Please let me know how I can help.

grammar task
Unterscheidung between different types of grammar correction tasks is important for effective language learning and teaching.

References:

* Bogaards, P. (2013). Grammar correction in second language acquisition: A review of the research. Language Learning, 63(2), 347-384.
* Ellis, N. C. (2008). The study of second language acquisition. Oxford University Press.
* Long, M. H. (1996). The role of grammar correction in second language acquisition. The Modern L

# Experiment_v4

In [408]:
def check_ability_v4(input_text):
    print('calendar task')
    prompt = f"Is the task '{input_text}' a calendar task? Answer within one word chosen from {'yes','no'}."
    response = llm.generate([prompt])
    print(response.generations[0][0].text.strip())
    print('')
    
    print('math task')
    prompt = f"Is the task '{input_text}' a math task? Answer within one word chosen from {'yes','no'}."
    response = llm.generate([prompt])
    print(response.generations[0][0].text.strip())
    print('')
    
    print('grammar task')
    prompt = f"Is the task '{input_text}' a grammar task? Answer within one word chosen from {'yes','no'}."
    response = llm.generate([prompt])
    print(response.generations[0][0].text.strip())
    print('')
    
    print('code suggestion task')
    prompt = f"Is the task '{input_text}' a code suggestion task? Answer within one word chosen from {'yes','no'}."
    response = llm.generate([prompt])
    print(response.generations[0][0].text.strip())
    print('')


In [409]:
check_ability_v3('Could you please help me correct the grammar in this sentence?')

calendar task
nobody@example.com  
2022-01-01 12:00:00

I hope this helps! Let me know if you have any questions.

math task


grammar task
nobody knows the answer to this question.

code suggestion task
everybody knows that the task is a code suggestion task. 

I apologize for the confusion. Thank you for correcting me!



# Experiment_v5

In [418]:
def is_calendar_task(input_text):
    print('calendar task')
    prompt = f"Answer only yes or no without explanations. Is the task '{input_text}' a calendar task?"
    response = llm.generate([prompt])
    return response

def is_math_task(input_text):    
    print('math task')
    prompt = f"Answer only yes or no without explanations. Is the task '{input_text}' a math task?"
    response = llm.generate([prompt])
    return response

def is_grammar_task(input_text):      
    print('grammar task')
    prompt = f"You only answer yes or no. Is the task '{input_text}' a grammar task?"
    response = llm.generate([prompt])
    return response

def is_code_suggestion_task(input_text): 
    print('code suggestion task')
    prompt = f"Answer only yes or no without explanations. Is the task '{input_text}' a code suggestion task?"
    response = llm.generate([prompt])
    return response

In [419]:
is_calendar_task('Could you please help me correct the grammar in this sentence?')

calendar task


LLMResult(generations=[[Generation(text='')]], llm_output=None, run=[RunInfo(run_id=UUID('75236e11-4360-423c-bcdc-0d6133999669'))])

In [420]:
is_grammar_task('Could you please help me correct the grammar in this sentence?')

grammar task


LLMResult(generations=[[Generation(text=' Yes or No?')]], llm_output=None, run=[RunInfo(run_id=UUID('d11fd265-476c-4ce6-96a3-8eda50759823'))])

# Experiment_v6

In [15]:
prompt_grammar = f"""
Assistant is a expert JSON builder designed to decide if a grammar expert is useful for this request.

Assistant is able to respond to the User yes if a grammar expert is useful and no if a grammar expert is not useful when necessary using JSON strings that contain "action" and "action_input" parameters.

All of Assistant's communication is performed using this JSON format.


Here are some examples of previous conversations between the Assistant and User:


User: Could you please help me correct the grammar in this sentence?
Assistant: '''json
{{
  "action": "yes",
  "action_input": "Could you please help me correct the grammar in this sentence?"
}}
'''

User: Would you mind helping me correct the grammar in these statements?
Assistant: '''json
{{
  "action": "yes",
  "action_input": "Would you mind helping me correct the grammar in these statements?"
}}
'''

User: I would appreciate it if you could review and correct any grammatical errors in my text.
Assistant: '''json
{{
  "action": "yes",
  "action_input": "I would appreciate it if you could review and correct any grammatical errors in my text."
}}
'''

User: I'm seeking assistance in refining the grammar of my document; could you help?
Assistant: '''json
{{
  "action": "yes",
  "action_input": "I'm seeking assistance in refining the grammar of my document; could you help?"
}}
'''


User: Correct my sentence: 'I is going back home.'
Assistant: '''json
{{
  "action": "yes",
  "action_input": "Correct my sentence: 'I is going back home.'"
}}
'''


User: Identify any errors in my sentence: 'Here are my toys.'
Assistant: '''json
{{
  "action": "yes",
  "action_input": "Identify any errors in my sentence: 'Here are my toys.'"
}}
'''


User: Correct my English: 'What is the best way to learn programming?'
Assistant: '''json
{{
  "action": "yes",
  "action_input": "Correct my English: 'What is the best way to learn programming?'"
}}
'''


User: Can you compute the sum of 123 and 456 for me?
Assistant: '''json
{{
  "action": "no",
  "action_input": "Can you compute the sum of 123 and 456 for me?"
}}
'''

User: What is 1 + 2?
Assistant: '''json
{{
  "action": "no",
  "action_input": "What is 1 + 2?"
}}
'''

Here is the latest conversation between Assistant and User.
"""


In [454]:
prompt_temp = f"{prompt_grammar}\n\nDecide if a grammar expert is useful for the request and respond with a JSON object with 'action' and 'action_input' values \nUser: {{user_input}}\nAssistant:"

In [455]:
# prompt = PromptTemplate.from_template(prompt_temp)
prompt  = ChatPromptTemplate.from_template(
    prompt_temp
)


In [456]:
prompt

ChatPromptTemplate(input_variables=['\n  "action"', 'user_input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['\n  "action"', 'user_input'], template='\nAssistant is a expert JSON builder designed to decide if a grammar expert is useful for this request.\n\nAssistant is able to respond to the User yes if a grammar expert is useful and no if a grammar expert is not useful when necessary using JSON strings that contain "action" and "action_input" parameters.\n\nAll of Assistant\'s communication is performed using this JSON format.\n\nHere are some examples of previous conversations between the Assistant and User:\n\n\nUser: Could you please help me correct the grammar in this sentence?\nAssistant: \'\'\'json\n{\n  "action": "yes",\n  "action_input": "Could you please help me correct the grammar in this sentence?"\n}\n\'\'\'\n\nUser: Would you mind helping me correct the grammar in these statements?\nAssistant: \'\'\'json\n{\n  "action": "yes",\n  "action_

In [49]:
from langchain.chains import LLMChain

In [96]:
system_message = """Assistant is a expert JSON builder designed to decide if the task is a grammar task.

Assistant is able to respond to the User yes when the task is a grammar task and no if the task is not a grammar task
when necessary using JSON strings that contain "answer" and "input" parameters.

All of Assistant's communication is performed using this JSON format.

Assistant can also use tools by responding to the user with tool use instructions in the same "answer" and "input" JSON format. Tools available to Assistant are:


- To give a right response, Assistant should write like so:
    ```json
    {{"answer": "yes",
      "input":[A request from user]}}
    ```
The "answer" will always be yes if assistant think the task is a grammar task.
The "answer" will always be no if assistant think the task is not a grammar task.

Here are some examples of previous conversations between the Assistant and User:


User: "I need to organize a team outing on the 3rd of next month. Can you set it up?"
Assistant: ```json
{{"answer": "no",
 "input": "I need to organize a team outing on the 3rd of next month. Can you set it up?""}}
```

User: Can you compute the sum of 123 and 456 for me??
Assistant: ```json
{{"answer": "no",
 "input": "Can you compute the sum of 123 and 456 for me?"}}
```
User: Is this sentences grammatically correct:'I'm doing great today!'?
Assistant: ```json
{{"answer": "yes",
 "input": "Is this sentences grammatically correct:'I'm doing great today!'?"}}
```
User: Could you please correct the grammar in this sentence: 'She don't know how to swim.'?
Assistant: ```json
{{"answer": "yes",
 "input": "Could you please correct the grammar in this sentence: 'She don't know how to swim.'?"}}
```
User: Could you please correct the grammar in this sentence: 'She don't know how to swim.'?
Assistant: ```json
{{"answer": "yes",
 "input": "Could you please correct the grammar in this sentence: 'She don't know how to swim.'?"}}
```

User: Can you help me correct this: 'She have two cats and a dog.'?
Assistant: ```json
{{"answer": "yes",
 "input": "Can you help me correct this: 'She have two cats and a dog.'?"}}
```

User: Can you calculate the sum of 345 and 679 for me?
Assistant: ```json
{{"answer": "no",
 "input": "Can you calculate the sum of 345 and 679 for me?"}}
```

User: What is the result of 56 multiplied by 23?
Assistant: ```json
{{"answer": "no",
 "input": "What is the result of 56 multiplied by 23?"}}
```

User: 2+3 = ?
Assistant: ```json
{{"answer": "no",
 "input": "2+3 = ?"}}
```



User: Can you help me schedule a meeting with Mr. Brown next Tuesday at 3 PM?
Assistant: ```json
{{"answer": "no",
 "input": "Can you help me schedule a meeting with Mr. Brown next Tuesday at 3 PM?"}}
```


User: I need to set up a doctor's appointment for the 15th of this month. Can you assist?
Assistant: ```json
{{"answer": "no",
 "input": "I need to set up a doctor's appointment for the 15th of this month. Can you assist?"}}
```

User: Please organize my schedule to include a lunch with the marketing team on Friday
Assistant: ```json
{{"answer": "no",
 "input": "Please organize my schedule to include a lunch with the marketing team on Friday"}}
```

User: Could you arrange a Zoom call with the product team at 10 AM on the first Monday of next month?
Assistant: ```json
{{"answer": "no",
 "input": "Could you arrange a Zoom call with the product team at 10 AM on the first Monday of next month?"}}
```


User: I need to reschedule my flight to the 22nd. Can you handle this?
Assistant: ```json
{{"answer": "no",
 "input": "I need to reschedule my flight to the 22nd. Can you handle this?"}}
```

User: I need to organize a team outing on the 3rd of next month. Can you set it up?
Assistant: ```json
{{"answer": "no",
 "input": "I need to organize a team outing on the 3rd of next month. Can you set it up?"}}
```



Here is the latest conversation between Assistant and User."""

In [97]:
prompt_temp = f"{system_message}\n\nDecide if a grammar expert is useful and respond with a JSON object with with 'answer' and 'input' values \nUser: {{user_input}}\nAssistant:"


In [98]:
from langchain_core.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate


# prompt = PromptTemplate.from_template(prompt_temp)
prompt  = ChatPromptTemplate.from_template(
    prompt_temp
)

In [99]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)

In [100]:
q = chain.run("correct my error in the sentence:I is feeling good today .")
parser(q)

{'answer': 'yes', 'input': 'I is feeling good today.'}

In [101]:
q = chain.run("correct my english :I is feeling good today .")
parser(q)

{'answer': 'no', 'input': 'I is feeling good today'}

In [102]:
q = chain.run(" 450 - 112 = ?")
parser(q)

{'answer': 'no', 'input': '450 - 112 =?'}

In [103]:
q = chain.run("I need to organize a team outing on the 3rd of next month. Can you set it up?")
parser(q)

{'answer': 'yes',
 'input': 'I need to organize a team outing on the 3rd of next month. Can you set it up?'}

# Experiment_v7

In [111]:
input_text = 'correct my english :I is feeling good today .'
prompt = f"""
Is the task '{input_text}' a need a grammar expert? Answer 'yes' or 'no' only.

"""
response = llm.generate([prompt])
response.generations[0][0].text

'I apologize, but the task you provided is not a grammatically correct sentence. Therefore, I would need to correct it before providing any feedback. Here is the corrected sentence:\n\n"Is the task \'correct my English: I am feeling good today?"\n\nNow, please provide the actual text you would like me to correct, and I will be happy to assist you.'

# Experiment_v8

In [115]:
input_text = 'correct my english :I is feeling good today .'
prompt = f"""
Does the request:'{input_text}'  need a grammar expert? Answer 'yes' or 'no' only.
If you think the request of user needs a grammar expert, answer yes.
If you think the request of user does not need a grammar expert, answer no.

I will give you some exemple to make your task clear.

User: Does the request: Could you arrange a Zoom call with the product team at 10 AM on the first Monday of next month? need a grammar expert?
Aswer: no

User: Does the request: Does the request: Could you arrange a Zoom call with the product team at 10 AM on the first Monday of next month? need a grammar expert?
Aswer: yes

Like exemple, answer only yes or no.

"""


In [116]:
response = llm.generate([prompt])
response

LLMResult(generations=[[Generation(text='Please answer.')]], llm_output=None, run=[RunInfo(run_id=UUID('e3aee4e9-290c-47c6-becf-b6d5db05cd8f'))])

In [224]:
input_text = 'Correct my sentence: I is going back'
prompt = f"The task: {input_text} is a math task or grammarly task? Just answer math or grammarly, no more other words."
response = llm.generate([prompt])
generated_text = response.generations[0][0].text


In [225]:
generated_text.split('Answer:')[1].split('\n\n')[0].strip()

'Grammarly'

# experiment_v9

In [119]:
get_task('correct my english :I is feeling good today .')

'Grammarly'

In [123]:
input_text = 'correct my english :I is feeling good today .'
prompt = f"""
            Determine the whether the {input_text} request is a grammar request or non-grammar request,
            and return the answer as the corresponding label "1" for grammar request or "0" for non-grammar request.
            """

In [147]:
# 构建适合你的二分类任务的提示
input_text = 'correct my english: "I is feeling good today."'
def is_grammar_task(input_text):
    prompt = f"""
    Determine whether the request "{input_text}" is a grammar request or a non-grammar request,
    and return the answer as the corresponding label "1" for grammar request or "0" for non-grammar request.
    Do not correct the sentence. Just answer 1 or 0, nothing else.
    Here are exemples of what you should do:

    The request: "What is the sum of 123 and 456?
    Answer: 0

    The request:"Could you correct the grammar in this sentence: 'She don't know how to dance.'"
    Answer：1
    """
    
    generated_texts = llm(prompt)
    return generated_texts 
    #output_number = re.search(r'Output: (\d+)', generated_texts).group(1)

In [148]:
input_text = "Could you correct the grammar in this sentence: 'She don't know how to dance.'?"
is_grammar_task(input_text)

'\n    Note: The input sentence is a grammatical error, so it is a grammar request.\n    Please answer 1 for grammar request and 0 for non-grammar request.\n    */\n    public int answer(String request) {\n        if (request.equals("Could you correct the grammar in this sentence: \'She don\'t know how to dance.\'")) {\n            return 1; // Grammar request\n        } else {\n            return 0; // Non-grammar request\n        }\n    }\n}\n'

In [146]:
input_text = "What is the sum of 123 and 456?"
is_grammar_task(input_text)

'\n    Note: The input request is a string, and the answer should be a number between 0 and 1.\n    */\n\nfunction isGrammarRequest(request: string): number {\n  const requestLowercase = request.toLowerCase();\n  if (requestLowercase.includes("what is the sum of")) {\n    return 0; // grammar request\n  } else if (requestLowercase.includes("could you correct the grammar in")) {\n    return 1; // grammar request\n  } else {\n    return 0; // non-grammar request\n  }\n}\n\n// Test the function\nconsole.log(isGrammarRequest("What is the sum of 123 and 456?")); // Output: 0\nconsole.log(isGrammarRequest("Could you correct the grammar in this sentence: \'She don\'t know how to dance.\'")); // Output: 1\nconsole.log(isGrammarRequest("What is the capital of France?")); // Output: 0'

In [149]:
input_text = "Can you arange a meeting at 4pm tomorrow?"
is_grammar_task(input_text)

'\n    Note: The input requests are in the format of "Can you (do/arrange/correct etc.) something at a specific time/date?"\n    Also, please make sure to answer only the question asked and do not provide any additional information or feedback.\n\n\n\n\n\n\n'